In [ ]:
pip install --upgrade fpl

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pip install paho-mqtt

In [4]:
import asyncio
import json
import aiohttp
from fpl import FPL
import pandas as pd
import json
import requests
import re
import paho.mqtt.client as paho

In [ ]:
async def main():
    def points(player):
#         print(player.web_name)
#         print(player.event_points)
        
        return player.event_points
            
#     def bonus(event):
    async def subcheck(j,pick,picks,d,m,f):
        id=pick['element']
        response = requests.get('https://fantasy.premierleague.com/api/element-summary/'+str(id)+'/',verify=False)
#         print(json.loads(response.content)['history'][-1]['fixture'])
        fixtureid=json.loads(response.content)['history'][-1]['fixture']
        pickminutes=json.loads(response.content)['history'][-1]['minutes']
        fixture=await fpl.get_fixture(fixtureid)
#         print(fixture.started)
#         print((fixture.started)&(pickminutes==0))
        if((fixture.started) & (pickminutes==0)):
#             print(pickminutes)
            p=await fpl.get_player(pick['element'])
            pos=p.element_type            
            if(pos!=1):
                for i in range(3):
                    subid=picks[12+i]['element']
                    subresponse=requests.get('https://fantasy.premierleague.com/api/element-summary/'+str(subid)+'/',verify=False)
                    
                    subfixtureid=json.loads(subresponse.content)['history'][-1]['fixture']
                    subminutes=json.loads(subresponse.content)['history'][-1]['minutes']
                    subfixture=await fpl.get_fixture(subfixtureid)
                    subp=await fpl.get_player(subid)
                    subpos=subp.element_type
                    if(not((subfixture.started) & (subminutes==0))):
#                         print(pos)
#                         print(((pos==2)&(d==3)) or ((pos==3)&(m==2)) or ((pos==4)&(f==1)))
                        if(((pos==2)&(d==3)) or ((pos==3)&(m==2)) or ((pos==4)&(f==1))):
                            if(subpos==pos):
                                picks[j], picks[12+i]=picks[12+i], picks[j]
#                                 print("working")
                                break
                        else:
                            picks[j], picks[12+i]=picks[12+i], picks[j]
                            if(pos!=subpos):
                                if(pos==2):
                                    d=d-1
                                if(pos==3):
                                    m=m-1
                                if(pos==4):
                                    f=f-1
                                if(subpos==2):
                                    d=d+1
                                if(subpos==3):
                                    m=m+1
                                if(subpos==4):
                                    f=f+1
                            break
            else:
                picks[j], picks[11]=picks[11], picks[j]
                
                    
                
            
        
        
    async with aiohttp.ClientSession() as session:
        currentgw=1
        fpl = FPL(session)
        await fpl.login('userid', 'password')
        response = requests.get('https://fantasy.premierleague.com/api/fixtures/?event='+str(currentgw),verify=True)
#         print(response.text)
        gwfixturedata=json.loads(response.content)
        ids=[42113, 3750584, 840827, 10929, 339474]
        nicks=['Nandy', 'Soumya', 'Ari', 'Chow', 'Giri']
        bonus1=[]
        bonus2=[]
        bonus3=[]
        userdf=pd.DataFrame(data=ids).rename(columns={0: "ID"})
        userdf['Name']=''
        userdf['GW Points']=0
        for f in gwfixturedata:
            fixture=await fpl.get_fixture(f['id'])
            if((fixture.started)&(((len(fixture.get_bonus()['a'])+len(fixture.get_bonus()['h']))==0))):
    #             print(fixture.stats)
                bonus=sorted((fixture.stats['bps']['a'] + fixture.stats['bps']['h']), key = lambda i: i['value'],reverse=True)
                bi=3
                ai=3
                last=0
                for b in bonus:
                    if(b['value']!=last):
                        ai=bi
                    last=b['value']
                    if(ai<1):
                        break
                    if(ai==3):
                        bonus3.append(b)
                    if(ai==2):
                        bonus2.append(b)
                    if(ai==1):
                        bonus1.append(b)
                    
                    bi=bi-1

        print(bonus2)
        i=0
        picksarray=[]
        for id in ids: 
            user=await fpl.get_user(id)
            responsep = requests.get('https://fantasy.premierleague.com/api/entry/'+str(id)+'/event/'+str(currentgw)+'/picks/',verify=True)
#             print(json.loads(responsep.content))
            userdf.loc[userdf['ID']==id, 'Total Points']=json.loads(responsep.content)['entry_history']['total_points']
            picks = await user.get_picks(currentgw)
#             userdf.loc[userdf['ID']==id, 'Name']=user.player_first_name+' '+user.player_last_name
            userdf.loc[userdf['ID']==id, 'Name']=nicks[i]
#             print(user)
            m=0
            f=0
            d=0
            for pick in picks[currentgw]:
                player=await fpl.get_player(pick['element'])
                if(player.element_type==2):
                    d=d+1
                if(player.element_type==3):
                    m=m+1
                if(player.element_type==4):
                    f=f+1
                if(m+f+d==10):
                    break
            point=0
            captainpoints=0
            vcpoints=0
            for j in range(11):
                pick=picks[currentgw][j]
                player=await fpl.get_player(pick['element'])
                await subcheck(j,pick, picks[currentgw],d,m,f)
                pick=picks[currentgw][j]
                player=await fpl.get_player(pick['element'])
                p=points(player)
                if(any(b.get('element') == pick['element'] for b in bonus3)):
                    p=p+3
                if(any(b.get('element') == pick['element'] for b in bonus2)):
                    p=p+2
                if(any(b.get('element') == pick['element'] for b in bonus1)):
                    p=p+1                    
#                 print(picks[currentgw])        
#                 team=await fpl.get_team(player.for pick in picks[currentgw]:team)
#                 gw=await fpl.get_gameweek(1)
                if(pick['is_captain'] ):
                    captainpoints=p
                if(pick['is_vice_captain'] ):
                    vcpoints=p
                point=point+p
#                 print(point)
            if(captainpoints==0):
                point=point+vcpoints
            else:
                point=point+captainpoints
#             point=point-user.entry.event_transfers_cost
            extra_transfer=2
            transfergw=currentgw
            transfercount=0
            event_transfers=0
            usertransfers=await user.get_transfers()
            for t in usertransfers:
#                 print(t)
                if(t['event']==currentgw):
                    event_transfers=event_transfers+1
                else:
                    if(transfergw-t['event']>1):
                        extra_transfer=1
                        transfergw=t['event']
                        break
                    if(t['event']==transfergw):
                        transfercount=transfercount+1
                        extra_transfer=0;
                        transfergw=t['event']
                        break;
                    transfergw=t['event']
            if(event_transfers==0):
                event_transfers=1
            if(extra_transfer==0):
#                 print(user)
                point=point-((event_transfers-1)*4)
            if(extra_transfer==1):
                if(event_transfers==1):
                    event_transfers=2
                point=point-((event_transfers-2)*4)                
            userdf.loc[userdf['ID']==id, 'GW Points']=point
            userdf.loc[userdf['ID']==id, 'Total Points']=userdf.loc[userdf['ID']==id, 'Total Points']+point
#             if(id==173257):
#                 print(picks[1][0])
            i=i+1
        player= await fpl.get_player(375)
#         print(dir(player))
        return(userdf)

# await main()

# loop = asyncio.get_event_loop()
notfirstiter=False
olduserdf=pd.DataFrame()
while True:
    change=False
    userdf=await main()
    print(notfirstiter)
    print(olduserdf)
    print(userdf )
    if(notfirstiter):
        for index,row in  userdf.iterrows():
            if(row['GW Points']-olduserdf.iloc[index]['GW Points']>2):
                change=True
                break
    if((change)):
        print("sending")
        client = paho.Client()
        client.connect("broker.hivemq.com",1883)
        client.publish("FPL",userdf.to_string(), qos=2)
    else:
        print("not sending")
    olduserdf=userdf
    notfirstiter=True